Fuente: https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/

In [2]:
!pip install textacy

     |████████████████████████████████| 204kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 12.7MB/s eta 0:00:01
     |████████████████████████████████| 194kB 10.3MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 16.0MB/s eta 0:00:01
     |████████████████████████████████| 92kB 6.7MB/s  eta 0:00:01
     |████████████████████████████████| 122kB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 7.9MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 7.4MB/s eta 0:00:01
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp37-cp37m-linux_x86_64.whl size=87398 sha256=936e36bb4908a2478bd6ddec9ef654160f57cccc95e0ffe3c60db36f70da9b0d
  Stored in directory: /home/jovyan/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
  Created wheel for pyemd: filename=pyemd-0.5.1-cp37-cp37m-linux_x86_64.whl size=379141 sha256=e50f6a88fff6568911d5cbee63

In [3]:
# Importando las librerías que vamos a utilizar
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import textacy
from textacy.datasets import Wikipedia
from collections import Counter, defaultdict
import warnings; warnings.simplefilter('ignore')

# graficos incrustados
%matplotlib inline

# función auxiliar
def leer_texto(texto):
    """Función auxiliar para leer un archivo de texto"""
    with open(texto, 'r') as text:
        return text.read()

In [3]:
# Descargando copus de wikipedia
wp = Wikipedia(data_dir='/home/jovyan/work/data', lang='es')
wp.download()

100%|██████████| 6.18G/6.18G [1:59:33<00:00, 862kB/s]   


In [4]:
# Chequeando la información descargada
wp.info

{'name': 'wikipedia',
 'site_url': 'https://en.wikipedia.org/wiki/Main_Page',
 'description': 'All pages for a given language- and version-specific Wikipedia site snapshot.'}

In [5]:
for text in wp.texts(min_len=1000, limit=2):
    print(text[:375], "\n")

El Hamelin Piscina Reserva de Naturaleza Marina es una reserva de naturaleza marina protegida localizado en el Patrimonio Mundial de la UNESCO –Bahía Shark listado en la región de Gascoyne en Australia Occidental. Las 127,000-hectárea (310,000-acres) de la reserva de naturaleza presume los ejemplos más diversos y abundantes de stromatolitos, o 'fósiles vivientes', en el mu 

The Future Sugar es el segundo álbum de estudio del grupo de rock mexicano Rey Pila, lanzado el 25 de septiembre de 2015 bajo el sello Cult Records.[8]​ El disco fue grabado en los estudios DFA de la ciudad de Nueva York y cuenta con la producción de Chris Coady y Julian Casablancas.

Todas las canciones por Diego Solórzano, Andrés Velasco y Rodrigo Blanco "Fire Away" – 3: 



In [4]:
# Detectando el idioma con taxtacy
saludos = ["Buenos días", "Hello", "Bonjour", "Guten Tag", "Buon giorno", "Bom dia"]
for saludo in saludos:
    print(textacy.lang_utils.identify_lang(saludo))

100%|██████████| 66.7M/66.7M [00:27<00:00, 2.45MB/s]


es
en
fr
da
it
pt


In [6]:
# Cargando el modelo en español de spacy
!python -m spacy download es
nlp = textacy.load_spacy_lang('es')

     |████████████████████████████████| 16.2MB 9.4MB/s eta 0:00:01
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172936 sha256=a1fc5bf2a98d1c08e4a35c1d698832d0ca9125a311f8f8dc2717e750cacfb4cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-qqrkh55h/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/es_core_news_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


In [7]:
texto = leer_texto('ORWELL_1984.txt')
texto_procesado = nlp(texto)

In [8]:
sentencias = [s for s in texto_procesado.sents]
print(len(sentencias))

6435


In [9]:
# imprimir las primeras 10 sentencias para verificar el texto
print(sentencias[1:11])

[Winston Smith, con la barbilla clavada en el pecho en su esfuerzo por burlar el molestísimo viento, se deslizó rápidamente por entre las puertas de cristal de las Casas de la Victoria, aunque no con la suficiente rapidez para evitar que una ráfaga polvorienta se colara con él.
, El vestíbulo olía a legumbres cocidas y a esteras viejas., Al fondo, un cartel de colores, demasiado grande para hallarse en un interior, estaba pegado a la pared., Representaba sólo un enorme rostro de más de un metro de anchura: la cara de un hombre de unos cuarenta y cinco años con un gran bigote negro y facciones hermosas y endurecidas., Winston se dirigió hacia las escaleras., Era inútil intentar subir en el ascensor., No funcionaba con frecuencia y en esta época la corriente se cortaba durante las horas de día., Esto era parte de las restricciones con que se preparaba la Semana del Odio., Winston tenía que subir a un séptimo piso., Con sus treinta y nueve años y una úlcera de varices por encima del tobil

In [10]:
# sentencias con las que aparece el Gran Hermano
[sent for sent in texto_procesado.sents if 'Gran Hermano' in sent.string][-10:]

[—¿Morirá el Gran Hermano?,
 No habrá lealtad; no existirá más fidelidad que la que se debe al Partido, ni más amor que el amor al Gran Hermano.,
 Pensó en el Gran Hermano.,
 ¿Cuáles eran sus verdaderos sentimientos hacia el Gran Hermano?,
 Dime: ¿cuáles son los verdaderos sentimientos que te inspira el Gran Hermano?,
 Tienes que amar al Gran Hermano.,
 » Winston miró el retrato del Gran Hermano.,
 Volvió a mirar el retrato del Gran Hermano.,
 Amaba al Gran Hermano.
 
 
 
 
 ,
 Hubiera sido posible, por ejemplo, decir el «Gran Hermano inbueno».]

In [30]:
def encontrar_personajes(doc):
    """
    NO FUNCIONA
    Devuelve una lista de los personajes de un `doc` con su cantidad de
    ocurrencias
    
    :param doc: NLP documento parseado por Spacy
    :return: Lista de Tuplas con la forma
        [('winston', 686), ("o'brien", 135), ('julia', 85),]
    """
    
    personajes = Counter()
    for ent in doc.ents:
        if ent.label_ == 'PER':
            personajes[ent.lemma_] += 1
            
    return personajes.most_common()


def encontrar_personajes_sin_counter(doc):
    """
    Devuelve una lista de los personajes de un `doc` con su cantidad de
    ocurrencias
    
    :param doc: NLP documento parseado por Spacy
    :return: Lista de Tuplas con la forma
        [('winston', 686), ("o'brien", 135), ('julia', 85),]
    """
    personajes = dict()
    for entity in texto_procesado.ents:
        if entity.label_ == 'PER':
            personajes[entity.lemma_] = personajes.get(entity.lemma_, 0) + 1

    pers_list = [(k, v) for k, v in personajes.items()]         
    pers_list = sorted(pers_list, key=lambda x: x[1], reverse=True)
    
    return pers_list


In [32]:
# Extrayendo los personajes principales del texto y contando cuantas veces son nombrados.
#print(encontrar_personajes(texto_procesado))
print(encontrar_personajes_sin_counter(texto_procesado)[:20])

[('Winston', 712), ('Julia', 135), ("O'Brien", 101), ('Había', 68), ('Parsons', 33), ('Goldstein', 28), ('Qué', 26), ('Charrington', 20), ('Cómo', 17), ('Aquí', 14), ('Oceanía', 13), ('Sabes', 13), ('Katharine', 13), ('Ampleforth', 12), ('Le', 11), ('Rutherford', 11), ('Aquello', 10), ('De', 10), ('Empezó', 10), ('Has', 10)]


In [37]:
def personaje_verbo(doc, verbo):
    """
    Encontrar los personajes que utilizan determinado `verbo` en `doc`
    
    :param doc: NLP documento parseado por Spacy
    :param verbo: un objeto String 
    :return: lista de personajes que utilizan `verbo`
    """
    verbos = list()
    for ent in texto_procesado.ents:
        if ent.label_ == 'PER' and ent.root.head.lemma_ == 'decir':
            verbos.append(ent.text)
    contar_verbo = Counter(verbos)
    
    return contar_verbo.most_common(10)

In [38]:
# Encontrar personajes que utilizan determinado verbo
personaje_verbo(texto_procesado, "dijo")

[('Winston', 54),
 ('Julia', 22),
 ("O'Brien", 20),
 ('Parsons', 4),
 ('Había', 2),
 ('Julia—.', 2),
 ('Goldstein', 1),
 ('Syme', 1),
 ('Oye', 1),
 ('fin—. Les', 1)]

In [39]:
# Trabajando con las entidades del texto
# Una entidad nombrada es cualquier objeto del mundo real como una persona,
# ubicación, organización o producto con un nombre propio.

# tipos de entidades del texto
set(ent.label_ for ent in texto_procesado.ents)

{'LOC', 'MISC', 'ORG', 'PER'}

In [40]:
# Entidades nombradas de tipo ORG
[ent for ent in texto_procesado.ents if ent.label_ == 'ORG'][:10]

[Partido,
 Partido,
 «,
 «,
 «,
 policía,
 Partido,
 Partido Interior,
 Partido Interior,
 Partido]

In [41]:
# Partes de la oración (POS)
# En las partes de la oración se etiquetan las palabras de acuerdo a lo 
# que significan segun su contexto. Algunas de estas etiquetas pueden
# ser: Adjetivos, verbos, adverbios, conjunciones, pronombres, sustantivos.

# Etiquetas del texto
set(token.pos_ for token in texto_procesado)

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SPACE',
 'VERB'}

In [42]:
# Etiquetas de tipo ADJ
[token.orth_ for token in texto_procesado if token.pos_ == 'ADJ'][1:11]

['luminoso',
 'frío',
 'clavada',
 'molestísimo',
 'suficiente',
 'polvorienta',
 'cocidas',
 'viejas',
 'grande',
 'pegado']

In [43]:
# Etiquetas de tipo PROPN
[token.orth_ for token in texto_procesado if token.pos_ == 'PROPN'][1:11]

['CAPITULO',
 'Winston',
 'Smith',
 'Casas',
 'Victoria',
 'hallarse',
 'Winston',
 'Semana',
 'Odio',
 'Winston']